In [3]:
from tst import TimeSeriesTransformer

## Model parameters
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 1 # The number of input variables. 1 if univariate forecasting.
dec_seq_len = 92 # length of input given to decoder. Can have any integer value.
enc_seq_len = 153 # length of input given to encoder. Can have any integer value.
output_sequence_length = 58 # Length of the target sequence, i.e. how many time steps should your forecast cover
max_seq_len = enc_seq_len # What's the longest sequence the model will encounter? Used to make the positional encoder

model = TimeSeriesTransformer(
    dim_val=dim_val,
    batch_first=True,
    input_size=input_size, 
    dec_seq_len=dec_seq_len,
    out_seq_len=output_sequence_length, 
    n_decoder_layers=n_decoder_layers,
    n_encoder_layers=n_encoder_layers,
    n_heads=n_heads)

c:\Users\zpulliam\Anaconda3\envs\ds\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset


class SP_500(Dataset):
    def __init__(self, frequency, splits):
        # frequency = daily, weekly, or monthly (from 'frequency'_prices folder)
        # splits = number of times a single cvs file will be split

        self.frequency = frequency
        self.folder = frequency + '_prices'
        self.splits = splits

        self.files = os.listdir(self.folder)
        new_files = []

        # set max file length ( 5 years of data )
        max = 0
        for file in self.files:
            data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
            if len(data.index) > max:
                max = len(data.index)

        # remove files with less than 5 years of data
        for file in self.files:
            data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
            if len(data.index) == max:
                new_files.append(file)

        self.files = new_files
        self.data = []   # stores lists of csv and partition: [AAPL.csv, n]

        # create partitions list: [AAPL.csv, n]
        for file in self.files:
            for i in range(self.splits):
                self.data.append([file, i])


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        file = self.data[idx][0]        # cvs to read from
        partition = self.data[idx][1]   # partion based on number of splits

        data = pd.read_csv(os.path.join(self.folder, file), index_col=0)
        data_split = np.array_split(data, self.splits)

        # ensure all splits are of equal length
        if data_split[0].shape[0] != data_split[-1].shape[0]:
            for i, sub in enumerate(data_split):
                if sub.shape[0] != data_split[-1].shape[0]:
                    data_split[i] = data_split[i][0:-1]

        data = data_split[partition]

        x = data[['Open', 'High', 'Low', 'Volume', 'Close']]    # data

        mins, maxs = x.min(), x.max()                           # values for normalization

        x = (x-x.min())/(x.max()-x.min())

        x = x.to_numpy()
        mins = mins.to_numpy()
        maxs = maxs.to_numpy()

        return x, mins, maxs

In [92]:
from torch import nn, optim

from torch.utils.data import DataLoader
from rnn import LSTM, GRU

model = LSTM(input_dim=5, hidden_dim=32, output_dim=1, num_layers=2)

dataset = SP_500(4)
dataloader = DataLoader(dataset=dataset, batch_size=2, shuffle=True, num_workers=0)

lookback = 100

for i, data in enumerate(dataloader):
    #print(data[0].shape)

    seqs = []

    for i in range(data[0].shape[1]-1 - lookback): 
        seqs.append([data[0][:,i: i + lookback, :], data[1][:, i+1: i+1 + lookback, :]])

    print(len(seqs), seqs[0][0].shape, seqs[0][1].shape) # num sequences, train, target

213 torch.Size([2, 100, 5]) torch.Size([2, 100, 1])
213 torch.Size([2, 100, 5]) torch.Size([2, 100, 1])


In [ ]:
import torch

x = torch.rand(2, 5, 3)
y = torch.rand(2, 5, 3)



In [4]:
class poop():
    def __init__(self, x, y):
        self.z = 34
        self.a = x
        self.b = y
    
    def otherfunction():
        return 0

    def addition(self, i, j):
        k = self.a + self.b
        return k

first_class = poop(2, 3)

first_class.addition(9, 8)

17

In [1]:
import torch

torch.cuda.is_available()

True